# Causal Inference Examples
# 1 Foundations
Julian Hsu
Date Made: 5 Aug 2021 

### Table of Contents with Navigation Links
* [Write Causal Models](#Section1)
* [Simulate Data](#Section2)
* [Bootstrapping Examples](#Section3)
* [Bootstrapping Examples - unconfoundedness violation](#Section4)
* [Bootstrapping Examples - overlap violation](#Section5)


In [96]:
import pandas as pd
import numpy as np
import os as os 

from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib inline  

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete.conditional_models import ConditionalLogit

from IPython.display import display    


import scipy.stats 

from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import mean_squared_error


<a id='Section1'></a>

## Write Causal Models
Write several functions here for estimate HTE. Each model _must_ do datasplitting.
These functions will do a lot of predictions, so try to standardize the prediction models.


In [97]:
## Standardized Function for Predicting the Treatment Indicator.
def predict_treatment_indicator(dataset, split_name, n_data_splits, feature,treatment, model):
    treatment_hat = []
    for r in np.arange(n_data_splits):
        train = (dataset[split_name] != r)
        test = (dataset[split_name]==r)
        lg = model.fit(dataset[feature][train==True],dataset[treatment][train==True])
        prediction = lg.predict_proba(dataset[feature][test==True])[:,1]
        treatment_hat.extend(prediction)
    return np.array(treatment_hat)

## Standardized Function for Predicting Counterfactual Outcomes.
def predict_counterfactual_outcomes(dataset, split_name, n_data_splits, feature, treatment, outcome,model):
    yhat_treat = []
    yhat_control = []
    for r in np.arange(n_data_splits):
        train = (dataset[split_name] != r)
        test = (dataset[split_name]==r)            
        bin_control = (dataset[treatment]==0)
        bin_treat = (dataset[treatment]==1)        

        ## Predict counterfactual outcomes for treatment
        ols_treat=model.fit(dataset[feature][(bin_treat==True) & (train==True)], dataset[outcome][(bin_treat==True) & (train==True)]) 
        prediction = ols_treat.predict(dataset[feature][(test==True)])
        yhat_treat.extend(prediction)
        
        ## Predict counterfactual outcomes for control
        ols_control=model.fit(dataset[feature][(bin_control==True) & (train==True)], dataset[outcome][(bin_control==True) & (train==True)]) 
        prediction = ols_control.predict(dataset[feature][(test==True)])
        yhat_control.extend(prediction)
    return np.array(yhat_treat), np.array(yhat_control)



In [98]:
def ols_vanilla(data_est, 
                split_name, feature_name, outcome_name, treatment_name,
                ymodel,tmodel,
               n_data_splits, aux_dictionary ):
    ols = sm.OLS(data_est[outcome_name], sm.add_constant(data_est[[treatment_name]+feature_list]) ).fit()
    return ols.params[1], ols.bse[1]

In [99]:
def propbinning(data_est, 
                split_name, feature_name, outcome_name, treatment_name,
                ymodel,tmodel,
               n_data_splits,
               aux_dictionary):
        
    data_est[split_name] = np.random.choice(n_data_splits, len(data_est), replace=True)
    data_est = data_est.sort_values(by=split_name)
        
    ## Predict Treatment
    that = predict_treatment_indicator(data_est, split_name, n_data_splits, feature_name,treatment_name,tmodel)
    data_est['that'] = that
    
    ## Sort by the probabilities and split into bins.
    ## For each bin, estimate counterfactuals for the treatment and control groups.
    data_est['that_bin'] = pd.qcut(data_est['that'], q=aux_dictionary['n_bins'], labels=False)    
    min_size = data_est['that_bin'].value_counts().min()
    
    data_est = data_est.sort_values(by=[split_name,'that_bin'])
    
    yhat_treat = []
    yhat_control = []
    for b in range(aux_dictionary['n_bins']):
        ## Use the first and last entry of each bin as cutpoints.    
        bin_of_interest = (data_est['that_bin']==b)
        for r in np.arange(n_data_splits):
            train = (bin_of_interest==True) & (data_est[split_name] != r)
            test = (bin_of_interest==True) & (data_est[split_name]==r)            
            bin_control = (bin_of_interest==True) & (data_est[treatment_name]==0)
            bin_treat = (bin_of_interest==True) & (data_est[treatment_name]==1)        

            ## Predict counterfactual outcomes for treatment
            ols_treat=ymodel.fit(data_est[feature_name][(bin_treat==True) & (train==True)], data_est[outcome_name][(bin_treat==True) & (train==True)]) 
            tpred = ols_treat.predict(data_est[feature_name][(test==True) ])
            yhat_treat.extend(tpred)

            ## Predict counterfactual outcomes for control
            ols_control=ymodel.fit(data_est[feature_name][(bin_control==True) & (train==True)], data_est[outcome_name][(bin_control==True) & (train==True)]) 
            cpred = ols_control.predict(data_est[feature_name][(test==True)])
            yhat_control.extend(cpred)

    ## Take the difference between the counterfactuals
    treatment_estimate = np.array(yhat_treat) - np.array(yhat_control)
    
    ## Output the treatment estimate and propensity scores
    return np.average(treatment_estimate), np.std(treatment_estimate), that


**Inverse-propensity weighting** estimator, where asympotics come from
Hirano, Imbens, Ridder (2004) Econometrica: 
https://scholar.harvard.edu/imbens/files/efficient_estimation_of_average_treatment_effects_using_the_estimated_propensity_score.pdf

In [100]:
def ipw(data_est, 
                split_name, feature_name, outcome_name, treatment_name,
                ymodel,tmodel,
               n_data_splits,
               aux_dictionary):
    ## Create and sort by splits
    data_est[split_name] = np.random.choice(n_data_splits, len(data_est), replace=True)
    data_est.sort_values(by=split_name, inplace=True)

    ## 1st Stage: Predict treatment indicator
    that = predict_treatment_indicator(data_est, split_name, n_data_splits, feature_name,treatment_name,tmodel)

    keep_these = (that >= aux_dictionary['lower']) & (that <= aux_dictionary['upper'])    
    
    ipw_a = (data_est[outcome_name] / that)*(data_est[treatment_name]==1)
    ipw_b = (data_est[outcome_name] / (1-that))*(data_est[treatment_name]==0)
    
    
    return np.average(ipw_a[keep_these]) - np.average(ipw_b[keep_these]), np.ones(len(data_est))*-1, that,

In [101]:
def dml_plm(data_est, 
                split_name, feature_name, outcome_name, treatment_name,
                ymodel,tmodel,
               n_data_splits,
               aux_dictionary):

    ## Create and sort by splits
    data_est[split_name] = np.random.choice(n_data_splits, len(data_est), replace=True)
    data_est.sort_values(by=split_name, inplace=True)

    ## 1st Stage: Predict treatment indicator
    that = predict_treatment_indicator(data_est, split_name, n_data_splits, feature_name,treatment_name,tmodel)
        
    ## Residualize treatment and outcome    
    outcome_hat = []        
    for r in np.arange(n_data_splits):
        train = (data_est[split_name] != r)
        test = (data_est[split_name]==r)
        ols = ymodel.fit(data_est[feature_name][train==True],data_est[outcome_name][train==True])
        prediction = ols.predict(data_est[feature_name][test==True])
        outcome_hat.extend(prediction)

    outcome_hat = np.array(outcome_hat)
    
    treatment_residual = data_est[treatment_name].to_numpy() - that
    outcome_residual = data_est[outcome_name].to_numpy() - outcome_hat
    
    ## shave off propensity scores below and under certain thresholds
    keep_these = (that >= aux_dictionary['lower']) & (that <= aux_dictionary['upper'])    
    ## Second stage OLS, for a partial linear model
    X = sm.add_constant(treatment_residual[keep_these])
    finalmodel_fit = sm.OLS( list(outcome_residual[keep_these]), X).fit()
    
    return finalmodel_fit.params[-1], finalmodel_fit.bse[-1], that
        

In [102]:
def dml_irm(data_est, 
                split_name, feature_name, outcome_name, treatment_name,
                ymodel,tmodel,
               n_data_splits, aux_dictionary ):
    data_est[split_name] = np.random.choice(n_data_splits, len(data_est), replace=True)
    data_est = data_est.sort_values(by=split_name)

    ## 1st Stage: Predict treatment indicator
    that = predict_treatment_indicator(data_est, split_name, n_data_splits, feature_name,treatment_name,tmodel)
    
    ## 2nd Stage: Predict counterfactual outcomes
    yhat_treat, yhat_control = predict_counterfactual_outcomes(data_est, split_name, n_data_splits, feature_name, treatment_name, outcome_name,ymodel)
    
    ## Residualize:
    y_control_residual = data_est[outcome_name]- yhat_control
    y_treat_residual = data_est[outcome_name]- yhat_treat    
    
    ## IPWRA Estimator on the residuals
    ra_term = yhat_treat - yhat_control    

    first_fraction = (data_est[treatment_name]==1)*(y_treat_residual) / that
    second_fraction = (data_est[treatment_name]==0)*(y_control_residual) / (1-that)
    ipw_term = first_fraction - second_fraction
    
    treatment_estimate = ra_term + ipw_term
    keep_these = (that >= aux_dictionary['lower']) & (that <= aux_dictionary['upper'])    
        
    treatment_estimate = treatment_estimate[keep_these]
    
    ## standard error
    se_est = np.sqrt(np.var(treatment_estimate))
    
    return np.mean(treatment_estimate), se_est, that[keep_these]
    

<a id='Section2'></a>

## Bring in Simulated Data
Pretend we've never seen this data before, and do balance checks between treatment and control 

For fun, use the Friedman function: https://www.sfu.ca/~ssurjano/fried.html

In [103]:
def generate_data():
    N = 2000
    
    cov = [[1.00, 0.08, 0.05, 0.05],
           [0.08, 1.00,-0.08,-0.02],
           [0.05,-0.08, 1.00,-0.10],
           [0.05,-0.02,-0.10, 1.00]]
    cov = np.eye(4)
    X = np.random.multivariate_normal(np.zeros(4), cov,N)
    x1,x2,x3,x4= X[:,0],X[:,1],X[:,2],X[:,3]

    treatment_latent = 2*np.sin( np.pi * x4 * x3) + 10*(x2-0.5)**2 - 10*x1
    m,s = np.average(treatment_latent), np.std(treatment_latent)

    treatment_latent = (treatment_latent - m) / s
    
    random_t = np.random.normal(0,1,N)
    
    treatment_latent += random_t
    
    treatment = np.array( np.exp(treatment_latent) / (1+ np.exp(treatment_latent)) > np.random.uniform(0,1,N) ).astype(np.int32)

#     Y = 100 +0.5*x1 - 6*x2 + -2*x4*x1 + 0.5*x1*x2 - 7*(x3+1)**(0.5) + 8/(0.5+x3+x4)
    Y = 100 + 10*np.sin( np.pi * x1 * x2) + 20*(x3-0.5)**2 - 10*x4
#     GT = np.std(Y)
    random_y = np.random.normal(0,1,N)

    GT = 5
    Y += np.random.normal(1,2,N)
    Y += GT*(treatment==1) 
    
    df_est = pd.DataFrame({'x1':x1, 'x2':x2,'x3':x3,'x4':x4,'treatment':treatment, 'Y':Y, 'GT':GT} )
    df_est['x1_2'] = df_est['x1'].pow(2)
    df_est['x2_2'] = df_est['x2'].pow(2)
    df_est['x3_2'] = df_est['x3'].pow(2)
    df_est['x4_2'] = df_est['x4'].pow(2)    
    return df_est

In [104]:
model_max_iter = 500
## treatment prediction models
t_models = {}
t_models['LogitCV'] = LogisticRegressionCV(cv=5, random_state=27, n_jobs=-1)
t_models['logit'] = LogisticRegression(penalty='l2',solver='lbfgs', C=1, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L1_C2'] = LogisticRegression(penalty='l1',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['logit_L2_C5'] = LogisticRegression(penalty='l2',C=2, max_iter=model_max_iter, fit_intercept=True)
t_models['rf_md10'] = RandomForestClassifier(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
t_models['rf_md3'] = RandomForestClassifier(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
t_models['nn'] = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1,max_iter=model_max_iter)
## outcome prediction models
y_models = {}
y_models['LassoCV'] = LassoCV(cv=5, n_jobs=-1, normalize=True, random_state=27)
y_models['ols'] = LinearRegression()
y_models['lasso_a2'] = Lasso(alpha=2,max_iter=model_max_iter)
y_models['ridge_a2'] = Ridge(alpha=2,max_iter=model_max_iter)
y_models['rf_md10'] = RandomForestRegressor(n_estimators=25,max_depth=10, min_samples_split=200,n_jobs=-1)
y_models['rf_md3'] = RandomForestRegressor(n_estimators=25,max_depth=3, min_samples_split=200,n_jobs=-1)
y_models['nn'] = MLPRegressor(alpha=1e-5, hidden_layer_sizes=(3, 2), random_state=1, max_iter=model_max_iter)

In [105]:
n_data_splits = 4
aux_dictionary = {'n_bins': 2, 'n_trees':2, 'max_depth':2, 
                  'upper':0.999, 'lower':0.001,
                  'subsample_ratio':0.5}
bootstrap_number = 100


In [106]:
df = generate_data()

feature_list = [x for x in df.columns if 'x' in x]

ols = ols_vanilla(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
pbin = propbinning(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
plm = dml_plm(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
irm = dml_irm(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
ip = ipw(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )

In [ ]:
df = generate_data()
df['splits'] = np.random.choice(n_data_splits, len(df), replace=True)
df = df.sort_values(by='splits')    

## Predict Treatment
that = predict_treatment_indicator(df, 'splits', n_data_splits, feature_list,'treatment',t_models['LogitCV'])
df['that'] = that
fig,ax = plt.subplots(nrows=1,ncols=1, figsize=(9,3), sharex=True, sharey=True)
ax.hist(df.loc[df.treatment==1]['that'], density=False, facecolor='g', alpha=0.25)
ax.hist(df.loc[df.treatment==0]['that'], density=False, facecolor='b', alpha=0.25)
control_range_to_remove = np.percentile(df.loc[df.treatment==1]['that'], q= 50) , np.percentile(df.loc[df.treatment==1]['that'], q= 99)
print(control_range_to_remove)

df = df.loc[ (df.treatment==1) | ( (df.that.between(control_range_to_remove[0],control_range_to_remove[1])==False) & (df.treatment==0) )   ]
fig,ax = plt.subplots(nrows=1,ncols=1, figsize=(9,3), sharex=True, sharey=True)
ax.hist(df.loc[df.treatment==1]['that'], density=False, facecolor='g', alpha=0.25)
ax.hist(df.loc[df.treatment==0]['that'], density=False, facecolor='b', alpha=0.25)


<a id='Section4'></a>

## Bootstrapping
* Bootstrap results using random datasets when all three assumptions are satisfied.
* Bootstrap results when the unconfoundedness assumption is violated. Do this by removing one fot the features from training.
* Bootstrap results when the overlap assumption is violated. Do this by removing control observations with propensities near the median treatment obervation propensity.

In [107]:
ols_x = []
pbin_x= []
plm_x = []
irm_x = []
ipw_x = []

ols_x_unconf = []
pbin_x_unconf= []
plm_x_unconf = []
irm_x_unconf = []
ipw_x_unconf = []

ols_x_overlap = []
pbin_x_overlap= []
plm_x_overlap = []
irm_x_overlap = []
ipw_x_overlap = []

In [108]:
for b in range(bootstrap_number):
    df = generate_data()
    ols = ols_vanilla(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    pbin = propbinning(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    plm = dml_plm(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    irm = dml_irm(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    ip = ipw(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
    ols_x.append(ols[0])
    pbin_x.append(pbin[0])
    plm_x.append(plm[0])
    irm_x.append(irm[0])    
    ipw_x.append(ip[0])   
    
    ols = ols_vanilla(df, 
                    'splits', feature_list_ab, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    pbin = propbinning(df, 
                    'splits', feature_list_ab, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    plm = dml_plm(df, 
                    'splits', feature_list_ab, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    irm = dml_irm(df, 
                    'splits', feature_list_ab, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    ip = ipw(df, 
                'splits', feature_list_ab, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )
    ols_x_unconf.append(ols[0])
    pbin_x_unconf.append(pbin[0])
    plm_x_unconf.append(plm[0])
    irm_x_unconf.append(irm[0])    
    ipw_x_unconf.append(ip[0])        


    df['splits'] = np.random.choice(n_data_splits, len(df), replace=True)
    df = df.sort_values(by='splits')    
    ## Predict Treatment
    that = predict_treatment_indicator(df, 'splits', n_data_splits, feature_list,'treatment',t_models['LogitCV'])
    df['that'] = that    
    control_range_to_remove = np.percentile(df.loc[df.treatment==1]['that'], q= 50) , np.percentile(df.loc[df.treatment==1]['that'], q= 99)
    df = df.loc[ (df.treatment==1) | ( (df.that.between(control_range_to_remove[0],control_range_to_remove[1])==False) & (df.treatment==0) )   ]


    ols = ols_vanilla(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    pbin = propbinning(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    plm = dml_plm(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    irm = dml_irm(df, 
                    'splits', feature_list, 'Y', 'treatment',
                    y_models['LassoCV'],t_models['LogitCV'],
                   n_data_splits, aux_dictionary )
    ip = ipw(df, 
                'splits', feature_list, 'Y', 'treatment',
                y_models['LassoCV'],t_models['LogitCV'],
               n_data_splits, aux_dictionary )

    ols_x_overlap.append(ols[0])
    pbin_x_overlap.append(pbin[0])
    plm_x_overlap.append(plm[0])
    irm_x_overlap.append(irm[0])    
    ipw_x_overlap.append(ip[0])        

ols_x = np.array(ols_x) - 5
pbin_x = np.array(pbin_x) - 5
plm_x = np.array(plm_x) - 5
irm_x = np.array(irm_x) - 5
ipw_x = np.array(ipw_x) - 5

ols_x_unconf = np.array(ols_x_unconf) - 5
pbin_x_unconf = np.array(pbin_x_unconf) - 5
plm_x_unconf = np.array(plm_x_unconf) - 5
irm_x_unconf = np.array(irm_x_unconf) - 5
ipw_x_unconf = np.array(ipw_x_unconf) - 5    

ols_x_overlap = np.array(ols_x_overlap) - 5
pbin_x_overlap = np.array(pbin_x_overlap) - 5
plm_x_overlap = np.array(plm_x_overlap) - 5
irm_x_overlap = np.array(irm_x_overlap) - 5
ipw_x_overlap = np.array(ipw_x_overlap) - 5    


In [109]:
print(b)

99


In [110]:

print_avg_med_iqr(ols_x)    
print_avg_med_iqr(pbin_x)    
print_avg_med_iqr(plm_x)    
print_avg_med_iqr(irm_x)    
print_avg_med_iqr(ipw_x)    

print('')
print_avg_med_iqr(ols_x_unconf) 
print_avg_med_iqr(pbin_x_unconf)    
print_avg_med_iqr(plm_x_unconf)    
print_avg_med_iqr(irm_x_unconf)    
print_avg_med_iqr(ipw_x_unconf)    

print('')
print_avg_med_iqr(ols_x_overlap)    
print_avg_med_iqr(pbin_x_overlap)    
print_avg_med_iqr(plm_x_overlap)    
print_avg_med_iqr(irm_x_overlap)    
print_avg_med_iqr(ipw_x_overlap[~np.isnan(ipw_x_overlap)])    


AVG: 0.013  , MED: 0.012, IQR: 0.013 to 0.012
AVG: -0.047  , MED: -0.026, IQR: -0.047 to -0.026
AVG: -0.023  , MED: -0.034, IQR: -0.023 to -0.034
AVG: -0.017  , MED: -0.035, IQR: -0.017 to -0.035
AVG: 0.271  , MED: 0.783, IQR: 0.271 to 0.783

AVG: 0.013  , MED: 0.012, IQR: 0.013 to 0.012
AVG: -0.149  , MED: -0.166, IQR: -0.149 to -0.166
AVG: -0.006  , MED: -0.093, IQR: -0.006 to -0.093
AVG: 0.040  , MED: -0.123, IQR: 0.040 to -0.123
AVG: 0.656  , MED: 1.266, IQR: 0.656 to 1.266

AVG: 0.775  , MED: 0.754, IQR: 0.775 to 0.754
AVG: -0.203  , MED: 0.135, IQR: -0.203 to 0.135
AVG: 0.942  , MED: 0.919, IQR: 0.942 to 0.919
AVG: 2.455  , MED: 2.718, IQR: 2.455 to 2.718
AVG: 24.043  , MED: 25.236, IQR: 24.043 to 25.236
